# 1) Données

In [1]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%24)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = len(dataOperateurs.axes[0])
colsDataOperateurs = len(dataOperateurs.axes[1])
nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
dataPostes = pd.read_csv('./data/postes.csv',sep = ";" )
cardP = len(dataPostes.axes[0])
nomsPostes = dataPostes.iloc[0:cardO,1]
rho = [0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 1
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0]]

# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
sigma= [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1]]

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein
d = [3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]
delta = np.full((cardO, cardJ), 1)
for i in range(0,cardO):
    if d[i] != -1:
        d_i = d[i]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[i][j_s] = 0

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)
        
profils = np.zeros(cardO)
cardProfils = 0
for i in range(0,cardO):
    found = False
    for j in range(0,i):
        if not found:
            diff = False
            if d[i] == d[j]:
                for p in range(0,cardP):
                    if kappa[i][p] != kappa[j][p]:
                        diff = True
            else:
                diff = True
        
            if not diff:
                profil = profils[j]
                found = True
    if found:
        profils[i] = profil
    else:
        cardProfils = cardProfils + 1
        profils[i] = cardProfils

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
#print(delta)
#print(o)
#print(H[1])


# 2) Modèle mathématique


## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for i in range(0,cardO):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[i,j,p]=LpVariable(f"x({i,j,p})", cat=LpBinary) #'Continuous'

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for i in range(0,cardO):
    for r in range(0,cardR):    
        y[i,r]=LpVariable(f"y({i,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for i in range(0,cardO):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[i,s,p]=LpVariable(f"z({i,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for i in range(0,cardO):
    for s in range(0,cardS):    
        u[i,s]=LpVariable(f"u({i,s})", cat=LpBinary)

print("nb de variables x_ijp = ", len(x))
print("nb de variables y_ir  = ", len(y))
print("nb de variables z_isp = ", len(z))
print("nb de variables u_ir  = ", len(u))
print()
print("nb total de variables = ", len(x)+len(y)+len(z)+len(u))

nb de variables x_ijp =  4032
nb de variables y_ir  =  576
nb de variables z_isp =  576
nb de variables u_ir  =  24

nb total de variables =  5208


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModeleExact", LpMinimize)

prob += sum(u[i,s] for i in range(0,cardO) for s in range(0,cardS)), "obj"

## 2.3) Contraintes

In [4]:
#contrainte 1 Un opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for i in range(0,cardO):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[i,j,p] <= delta[i][j]*kappa[i][p]*o[j], f"C1_{i,j,p}")

#contrainte 2 Un opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[i,s,p] <= kappa[i][p], f"C2_{i,s,p}")

#contrainte 3 Chaque jour, l'opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for i in range(0,cardO):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                prob += x[i,j,p] <= z[i,s,p] + sum((1 - rho[p2])*z[i,s,p2] for p2 in range(0,cardP)), f"C3_{i,j,p}"

#contrainte 4 Chaque jour, un opérateur est affecté à au plus un poste

for i in range(0,cardO):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            prob += sum(x[i,j,p] for p in range(0,cardP)) == 1, f"C4_{i,j}"

#contrainte 5 Chaque jour, chaque poste est assuré par un opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            prob += sum(x[i,j,p] for i in range(0,cardO)) == 1, f"C5_{p,j}"
            
#contrainte 6 Chaque jour, un poste roulant est occupé par au plus une personne
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob += sum(x[i,j,p] for i in range(0,cardO)) <= 1, f"C52_{j,p}"

#contrainte 7 Chaque semaine, chaque opérateur est affecté à un poste

for i in range(0,cardO):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for p in range(0,cardP)) == 1, f"C6_{i,s}"

#contrainte 8 Chaque semaine, chaque poste est assuré par un opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for i in range(0,cardO)) == 1, f"C7_{s,p}"

#contrainte 9 Chaque opérateur est affecté à une position dans le roulement

for i in range(0,cardO):
    prob += sum(y[i,r] for r in range(0,cardR)) == 1, f"C8_{i}"

#contrainte 10 Chaque position dans le roulement est associée à un opérateur

for r in range(0,cardR):
    prob += sum(y[i,r] for i in range(0,cardO)) == 1, f"C9_{r}"

#contrainte 11 Un opérateur peut changer de poste si l'horaire de ce dernier est compatible

for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += z[i,s,p] <= sum(sigma[p][p2]*y[i,rFunction(s,p2)] for p2 in range(0,cardP)), f"C10_{i,s,p}"

#contrainte 12 Lorsqu'un jour un opérateur est affecté à un poste A alors qu'il est affecté à un poste B pour cette semaine, cela augmente son score d'insatisfaction
for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += u[i,s] >= (kappa[i][p]*rho[p]*y[i,rFunction(s,p)] - z[i,s,p]), f"C11_{i,s,p}"


## 2.4) Résolution du problème

In [5]:
import time

prob.writeLP("Modeles/ModeleExact.lp")

start = time.time() 
prob.solve(CPLEX_PY())
end = time.time()

duree = end - start
print()
print("Temps de résolution : " + str(duree))
print("Valeur objectif = ", value(prob.objective))

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 6159 rows and 1374 columns.
MIP Presolve modified 714 coefficients.
Reduced MIP has 3968 rows, 3834 columns, and 33117 nonzeros.
Reduced MIP has 3834 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (19.98 ticks)
Found incumbent of value 24.000000 after 0.09 sec. (49.34 ticks)
Probing changed sense of 144 constraints.
Probing time = 0.05 sec. (17.16 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 150 rows and 144 columns.
Reduced MIP has 3818 rows, 3690 columns, and 32685 nonzeros.
Reduced MIP has 3690 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.10 sec. (64.15 ticks)
Probing time = 0.02 sec. (5.11 ticks)
Clique table members: 16674.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to

# 3) Affichage de la solution

In [6]:
print("uis")
for s in range(0,cardS):
    for i in range(0,cardO):
        if(u[i,s].varValue != 0):
            print("u[",i,",",s,"] =", u[i,s].varValue)

print('Affectation aux roulements')
for i in range(0,24):
    for r in range(0,24):
        if(y[i,r].varValue != 0):
            print('y[',i,',',r,'] =', y[i,r].varValue)

print()
print('Affectations hebdomadaires')
for s in range(0,cardS):
    print('semaine',s)
    for i in range(0,cardO):
        for p in range(0,cardP):
            if z[i,s,p].varValue==1:
                print('semaine',s,',',i,'affecté au poste',p)
                
print('======================================================')

for p in range(0,cardP):
    print()
    if rho[p] == 1:
        print('poste',p,'non rouleur')
    else:
        print('poste',p,'rouleur')
    for j in range(0,cardJ):
        for i in range(0,cardO):
            if (x[i,j,p].varValue == 1):
                print(i,'occupe le poste',p,'le jour',j)
                
print('======================================================')

                
for i in range(0,cardO):
    print()
    print('planning de l\'opérateur',i)
    for j in range(0,cardJ):
        for p in range(0,cardP):
            if (x[i,j,p].varValue == 1):
                print('poste',p,'le jour',j)

uis
Affectation aux roulements
y[ 0 , 23 ] = 1.0
y[ 1 , 2 ] = 1.0
y[ 2 , 10 ] = 1.0
y[ 3 , 17 ] = 1.0
y[ 4 , 16 ] = 1.0
y[ 5 , 22 ] = 1.0
y[ 6 , 11 ] = 1.0
y[ 7 , 13 ] = 1.0
y[ 8 , 20 ] = 1.0
y[ 9 , 5 ] = 1.0
y[ 10 , 15 ] = 1.0
y[ 11 , 3 ] = 1.0
y[ 12 , 1 ] = 1.0
y[ 13 , 14 ] = 1.0
y[ 14 , 21 ] = 1.0
y[ 15 , 4 ] = 1.0
y[ 16 , 0 ] = 1.0
y[ 17 , 8 ] = 1.0
y[ 18 , 6 ] = 1.0
y[ 19 , 12 ] = 1.0
y[ 20 , 18 ] = 1.0
y[ 21 , 9 ] = 1.0
y[ 22 , 19 ] = 1.0
y[ 23 , 7 ] = 1.0

Affectations hebdomadaires
semaine 0
semaine 0 , 0 affecté au poste 23
semaine 0 , 1 affecté au poste 2
semaine 0 , 2 affecté au poste 10
semaine 0 , 3 affecté au poste 17
semaine 0 , 4 affecté au poste 16
semaine 0 , 5 affecté au poste 22
semaine 0 , 6 affecté au poste 11
semaine 0 , 7 affecté au poste 13
semaine 0 , 8 affecté au poste 20
semaine 0 , 9 affecté au poste 5
semaine 0 , 10 affecté au poste 15
semaine 0 , 11 affecté au poste 3
semaine 0 , 12 affecté au poste 1
semaine 0 , 13 affecté au poste 14
semaine 0 , 14 affe

# 4) Extraction des données au format texte

In [7]:
with open("solutionsTexte/solutionModeleExact", "w") as file_object:
        file_object.write(str(cardO) + "\n")           #cardO
        file_object.write(str(cardP) + "\n")           #cardP
        file_object.write(str(cardR) + "\n")           #cardR
        file_object.write(str(cardS) + "\n")           #cardS
        ligne = ""
        for p in range(0,cardP):
            ligne = ligne + str(rho[p])
        file_object.write(ligne + "\n")                #rho
        
        ligne = ""
        for j in range(0,cardJ):
            ligne = ligne + str(int(o[j]))
        file_object.write(ligne + "\n")                #o_j
        
        for i in range(0,cardO):
            ligne = ""
            for p in range(0,cardP):
                ligne = ligne + str(kappa[i][p])
            file_object.write(ligne + "\n")            #kappa_ip
            
        for p in range(0,cardP):
            ligne = ""
            for p2 in range(0,cardP):
                ligne = ligne + str(sigma[p][p2])
            file_object.write(ligne + "\n")            #sigma_pp'
            
        for i in range(0,cardO):
            ligne = ""
            for j in range(0,cardJ):
                ligne = ligne + str(delta[i][j])
            file_object.write(ligne + "\n")            #delta_ij
        
        for i in range(0,cardO):
            for j in range(0,cardJ):
                ligne = ""
                for p in range(0,cardP):
                    ligne = ligne + str(int(x[i,j,p].varValue))
                file_object.write(ligne + "\n")        #x_ijp
        
        for i in range(0,cardO):
            ligne = ""
            for r in range(0,cardR):
                ligne = ligne + str(int(y[i,r].varValue))
            file_object.write(ligne + "\n")            #y_ir

        for i in range(0,cardO):
            for s in range(0,cardS):
                ligne = ""
                for p in range(0,cardP):
                    ligne = ligne + str(int(z[i,s,p].varValue))
                file_object.write(ligne + "\n")        #z_isp
        
        file_object.write(str(int(value(prob.objective))) + "\n")

# 5) Extraction des données au format CSV

In [8]:
import csv
with open('CSV/planningExact.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if x[i,j,p].varValue == 1:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)

In [9]:
print("Fin de programme")

Fin de programme
